# Tarea 3 de Aprendizaje de máquina
Eva Rivarola
Pablo Barranco
J. Paul Virueña

#### Importamos librerías

In [69]:
# Import data
import warnings
warnings.simplefilter('ignore')
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

FLAGS = None
# Sólo informe sobre errores
tf.logging.set_verbosity(tf.logging.ERROR)

%matplotlib inline

In [70]:
import numpy as np

Importamos el data set.

In [71]:
dataset = pd.read_csv("wdbc.data", header = None).drop(columns=[0])

In [72]:
def one_hot(row):
    return 1 if row=='M' else 0

dataset[1] = dataset[1].apply(one_hot)

Ahora limpiamos los datos. Como no nos prove informacion la columna de ID, la quitaremos y la variable Diagnostico correspondiente a la columna 1, la cambiaremos de categorica a binaria.Cero si es benigno "B", uno en caso contrario "M"..

Ahora separamos nuestro conjunto de entrenamiento 80% del de prueba 20%.

In [73]:
#print(len(train), 'train examples')
#print(len(test), 'test examples')
train_set, test = train_test_split(dataset, test_size=0.2)
train_set = train_set.reset_index().drop(columns=['index'])
test = test.reset_index().drop(columns=['index'])
y_train = train_set[1].to_numpy()
x_train = train_set.drop(columns=[1]).to_numpy()
x_test = test.drop(columns = [1]).to_numpy()
y_test = test[1].to_numpy()

Definimos las variables y placeholders de nuestra modelo.

In [74]:
data = tf.placeholder(tf.float32, [None, 30]) 
A = tf.Variable(tf.zeros([30, 1])) 
b = tf.Variable(tf.zeros([1]))
mod = tf.matmul(data, A) + b
target = tf.placeholder(tf.float32, [None, 1])
init = tf.global_variables_initializer()
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels=target))

Entrenamos el modelo

In [75]:
session = tf.Session()
session.run(init)

In [76]:
learning_rate = 0.2
batch_size = 100
max_iters = 1500
goal = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [77]:
prediction = tf.round(tf.sigmoid(mod))
correct = tf.cast(tf.equal(prediction,target),dtype=tf.float32)
accuracy = tf.reduce_mean(correct)

In [78]:
loss_trace = []
train_accuracy = []
test_accuracy = []

In [79]:
for i in range(max_iters):
    bi = np.random.choice(len(x_train),size= batch_size)
    btx = x_train[bi]
    bty = np.matrix(y_train[bi]).T
    d = {data: btx, target:bty}

    session.run(goal,feed_dict=d)
    temp_loss = session.run(loss,feed_dict=d)

    temp_train_acc = session.run(accuracy,feed_dict={data:x_train, target:np.matrix(y_train).T})
    temp_test_acc = session.run(accuracy,feed_dict={data:x_test, target:np.matrix(y_test).T})
    loss_trace.append(temp_loss)
    train_accuracy.append(temp_train_acc)
    test_accuracy.append(temp_test_acc)
    if (i + 1) % 300 == 0:
        print('i: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(i + 1, temp_loss,
                                                                          temp_train_acc, temp_test_acc))


i:  300 loss: 1570.629028 train_acc: 0.903297 test_acc: 0.903509
i:  600 loss: 527.271545 train_acc: 0.916484 test_acc: 0.912281
i:  900 loss: 278.258392 train_acc: 0.923077 test_acc: 0.903509
i: 1200 loss: 1417.775879 train_acc: 0.857143 test_acc: 0.842105
i: 1500 loss: 1085.970581 train_acc: 0.920879 test_acc: 0.912281


In [80]:
# Test trained model
correct_prediction = tf.equal(mod, target) 
accuracy = tf.reduce_mean(correct) 
print(session.run(accuracy, feed_dict={data:x_test, target:np.matrix(y_test).T})) 

0.9122807


### Conclsusiones
El modelo predice con una certidumbre del 92%